In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Define paths
# Data from https://aqs.epa.gov/aqsweb/airdata/download_files.html
path15 = '../01_Resources/EPA_AQI_data/daily_aqi_by_county_2015.csv'
path16 = '../01_Resources/EPA_AQI_data/daily_aqi_by_county_2016.csv'
path17 = '../01_Resources/EPA_AQI_data/daily_aqi_by_county_2017.csv'
path18 = '../01_Resources/EPA_AQI_data/daily_aqi_by_county_2018.csv'
path19 = '../01_Resources/EPA_AQI_data/daily_aqi_by_county_2019.csv'
path20 = '../01_Resources/EPA_AQI_data/daily_aqi_by_county_2020.csv'

In [ ]:
# Read in data for six year of AQI data
df15 = pd.DataFrame(pd.read_csv(path15))
df16 = pd.DataFrame(pd.read_csv(path16))
df17 = pd.DataFrame(pd.read_csv(path17))
df18 = pd.DataFrame(pd.read_csv(path18))
df19 = pd.DataFrame(pd.read_csv(path19))
df20 = pd.DataFrame(pd.read_csv(path20))

df19

In [ ]:
# Append all six years into one DF
six_years_df = df15.append([df16,df17,df18,df19,df20], ignore_index=True)

In [ ]:
# Split "Date" column into "Year", "Month", and "Day" columns
six_years_df['Year'] = [d.split('-')[0] for d in six_years_df.Date]
six_years_df['Month'] = [d.split('-')[1] for d in six_years_df.Date]
six_years_df['Day'] = [d.split('-')[2] for d in six_years_df.Date]

# Create column to hold Month and Day w/o year
# This is to compare specific days across all six years
# TODO: Find a cleaner, more effective way to do this
six_years_df['Month_Day'] = six_years_df['Month'] + '-' + six_years_df['Day']

# Change type from str to int64
six_years_df = six_years_df.astype({'Year': 'int64','Month': 'int64','Day': 'int64'})

# Check result
# six_years_df

In [ ]:
# Limit DF to days before June
before_june_df = six_years_df.loc[six_years_df['Month']<6]

# Export combined-year DF to CSV file
before_june_df.to_csv('../01_Resources/EPA_AQI_data/AQI_data_2015_2020.csv')

In [ ]:
# Create DF to hold avg AQI for 2020 by day of the year
avg_2020 = six_years_df.loc[six_years_df['Year']==2020].groupby('Month_Day')['AQI'].mean().reset_index()

# Create DF to hold avg AQI for 2015-2019 by day of the year
avg_historic = six_years_df.loc[six_years_df['Month']<6]
avg_historic = avg_historic.loc[six_years_df.Year!=2020].groupby('Month_Day')['AQI'].mean().reset_index()

avg_2020

In [ ]:
# TODO: fix this viz
# Need to figure out how to correctly compare dates across years
avg_2020.plot(kind='line',figsize=(15,10))
plt.plot(avg_historic['Month_Day'], avg_historic['AQI'])
plt.legend(["2020", "Last 5 Years"])

In [ ]:
# Trying to figure out that sudden drop
df20.loc[df20['Date']=='2020-05-01']

In [ ]:
# Trying to figure out that sudden drop
df20.loc[df20['Date']=='2020-05-02']

In [ ]:
# Split "Date" column into "Year", "Month", and "Day" columns
df19['Year'] = [d.split('-')[0] for d in df19.Date]
df19['Month'] = [d.split('-')[1] for d in df19.Date]
df19['Day'] = [d.split('-')[2] for d in df19.Date]
# Change type from str to int64
df19 = df19.astype({'Year': 'int64','Month': 'int64','Day': 'int64'})

# Repeat for 2020 dataframe
df20['Year'] = [d.split('-')[0] for d in df20.Date]
df20['Month'] = [d.split('-')[1] for d in df20.Date]
df20['Day'] = [d.split('-')[2] for d in df20.Date]
df20 = df20.astype({'Year': 'int64','Month': 'int64','Day': 'int64'})

# Display result
df20

In [ ]:
# Exclude 2019 data beyond June
df19 = df19.loc[df19.Month < 6]
df20 = df20.loc[df20.Month < 6]

df19

In [ ]:
# Merge the DF on "County Code"
both_years = df19.append(df20, ignore_index=True)
both_years

In [ ]:
mean_aqi_19 = df19.groupby('Date')['AQI'].mean()
mean_aqi_20 = df20.groupby('Date')['AQI'].mean()

In [ ]:
mean_aqi_19.plot(kind='line',figsize=(17,12))
plt.plot(mean_aqi_20)
plt.ylabel('Avg. Air Quality Index')
plt.axvline(x='2020-02-25',c='red',alpha=.75)
plt.axvline(x='2020-05-01',c='g',alpha=.75)
plt.annotate('San Fransico declares state of emergency', xy=('2020-02-26', 65), c='r')
plt.annotate('Texas starts "reopening"', xy=('2020-04-08', 13), c='g')
plt.legend(["2019","2020"])
plt.title("2019 vs. 2020 Avg. Daily Air Quality Index")